In [1]:
pct = 'full'

In [2]:
!mkdir -p {pct}

In [3]:
!aws s3 sync s3://yifeim-code-data/criteo-raw/{pct} {pct} --exclude "*" --include "*.gz"

download: s3://yifeim-code-data/criteo-raw/full/val.txt.gz to full/val.txt.gz
download: s3://yifeim-code-data/criteo-raw/full/train.txt.gz to full/train.txt.gz
download: s3://yifeim-code-data/criteo-raw/full/test.txt.gz to full/test.txt.gz


In [4]:
!aws s3 sync s3://yifeim-code-data/criteo-raw/{pct} {pct} --exclude "*" --include "vocab.txt"

download: s3://yifeim-code-data/criteo-raw/full/vocab.txt to full/vocab.txt


In [5]:
%matplotlib inline
import pylab as pl

In [6]:
import numpy as np

In [7]:
import gzip

In [8]:
!gunzip < {pct}/train.txt.gz | head -n 10 

example 68965824: 57b4be06cf2b8d6c1f0679601ed1b76057b4be0790c591796bc73fcdc29c06db 0 2.973581e-02 1 12 1:300 2:600 3:0 4:47 5:1585 6:5 7:0 8:56 9:74 10:0
0 exid:68965824 11:1 12:0 13:0 14:0 15:0 16:4 23:55 24:321 25:155 32:90 35:204
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:11 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:77 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:77 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:10 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:40 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:0 14:0 15:0 24:320 25:154 35:203
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:10 24:320 25:151 34:24 35:201
0 exid:68965824 11:0 12:0 13:1 14:0 15:1 18:4 19:13 20:10 24:320 25:151 34:24 35:201

gzip: stdout: Broken pipe


In [9]:
import pandas as pd, mxnet as mx

In [10]:
vocab_df = pd.read_csv('{}/vocab.txt'.format(pct), header=None)

In [11]:
shape = len(vocab_df)+1

In [12]:
shape

84311

In [13]:
vocab_df['ind'] = range(1, shape)

In [14]:
str2ind = vocab_df.set_index(0)['ind'].to_dict()

In [15]:
import pickle

In [ ]:
for split in ['train','val','test']:
    with gzip.open('{}/{}.txt.gz'.format(pct, split), 'rt') as f:
        num_examples = 0
        
        for wcl, line in enumerate(f):
            line = line.strip('\n')
            
            if line.startswith('example'):
                num_examples += 1
            
    print(split, 'done', num_examples, wcl)

In [ ]:
def csr_vstack(*arr, dtype='float32', shape=None):
    data = []
    indices = []
    indptr = [0]

    shape0 = 0
    shape1 = arr[0].shape[1] if shape is None else shape[1]
    nnz = 0

    for i,a in enumerate(arr):
        if a is None:
            indptr.append(nnz)
            shape0 += 1
        else:
            if type(a) != ss.csr_matrix:
                a = ss.csr_matrix(a)

            assert a.shape[1] == shape1

            data   .extend(list(a.data))
            indices.extend(list(a.indices))
            indptr .extend([iptr+nnz for iptr in a.indptr[1:]])

            shape0 += a.shape[0]
            nnz += a.nnz

    if shape is None:
        shape = (shape0, shape1)

    csr = ss.csr_matrix((data, indices, indptr), shape=shape, dtype=dtype)
    return csr

In [ ]:
import tqdm

In [ ]:
import scipy.sparse as ss

In [27]:
def fold_dataset(split):
    record = mx.recordio.MXIndexedRecordIO(
        './{}/{}.idx'.format(pct, split),
        './{}/{}.rec'.format(pct, split),
        'w')
    
    with gzip.open('{}/{}.txt.gz'.format(pct, split), 'rt') as f:
        num_examples = 0
        
        A = []
        
        for line in tqdm.tqdm_notebook(f):
            line = line.strip('\n')
            
            if line.startswith('example'):
                if len(A):
                    A = csr_vstack(*A)
                    record.write_idx(int(idx[:-1]), pickle.dumps([
                        dense,
                        (x.data, x.indices, x.indptr, x.shape),
                        (A.data, A.indices, A.indptr, A.shape),
                        A.shape[0],
                        float(mu), float(rew)]))
                    A = []
                num_examples += 1
#                 if num_examples > 7117:
#                     break
                    
                _, idx, _, rew, mu, d1, d2, *keys = line.split(' ')
                dense = [int(d1), int(d2)]
                x = ss.csr_matrix((
                    [1./np.sqrt(len(keys))] * len(keys),
                    [str2ind.get(k,0) for k in keys],
                    [0, len(keys)]), shape=(1, shape))
                
            else:
                _, _, *keys = line.split(' ')
                A.append(ss.csr_matrix((
                    [1./np.sqrt(len(keys))] * len(keys),
                    [str2ind.get(k,0) for k in keys],
                    [0, len(keys)]), shape=(1, shape))
                )
                
        if len(A):
                    A = csr_vstack(*A)
                    record.write_idx(int(idx[:-1]), pickle.dumps([
                        dense,
                        (x.data, x.indices, x.indptr, x.shape),
                        (A.data, A.indices, A.indptr, A.shape),
                        A.shape[0],
                        float(mu), float(rew)]))
                    A = []
            
        record.close()
            
    print(split, 'done')

In [28]:
from multiprocessing import Pool

In [ ]:
with Pool() as p:
    p.map(fold_dataset, ['train','val','test'])

In [30]:
!wc -l {pct}/train.idx

7087738 full/train.idx


In [32]:
!aws s3 sync {pct} s3://yifeim-code-data/criteo-rec/{pct} --delete

upload: full/train.idx to s3://yifeim-code-data/criteo-rec/full/train.idx            6.5 GiB (87.5 MiB/s) with ~8 file(s) remaining (calculating...)
upload: full/test.idx to s3://yifeim-code-data/criteo-rec/full/test.idx               
upload: full/val.idx to s3://yifeim-code-data/criteo-rec/full/val.idx              calculating...)
upload: full/train.txt.gz to s3://yifeim-code-data/criteo-rec/full/train.txt.gz                                                                
upload: full/test.txt.gz to s3://yifeim-code-data/criteo-rec/full/test.txt.gz5 MiB/s) with 6 file(s) remainingremaining
upload: full/vocab.txt to s3://yifeim-code-data/criteo-rec/full/vocab.txt
upload: full/val.txt.gz to s3://yifeim-code-data/criteo-rec/full/val.txt.gzremainingremainingremainingremaining


upload: full/train.rec to s3://yifeim-code-data/criteo-rec/full/train.recngB/s) with 3 file(s) remaining (102.3 MiB/s) with 3 file(s) remainingB/48.1 GiB (102.3 MiB/s) with 3 file(s) remainingremainingremainingremainingB/s) with 3 file(s) remaining (102.3 MiB/s) with 3 file(s) remaininged 45.8 GiB/48.1 GiB (102.3 MiB/s) with 3 file(s) remainings) remaining3 file(s) remainingremainingremaining3 file(s) remainingremainingB/s) with 3 file(s) remainingremainingB/s) with 3 file(s) remainingB/48.1 GiB (102.3 MiB/s) with 3 file(s) remainingremainingremainingB/s) with 3 file(s) remainingremaining3 file(s) remaining (102.3 MiB/s) with 3 file(s) remainingremainingremaining3 file(s) remainingremainingremainingremainingremainingremainingremainingremainingremainingremainingremainingremainingremaining3 file(s) remaining (102.3 MiB/s) with 3 file(s) remainingremainingremainingremainingB/s) with 3 file(s) remainingremainingB/s) with 3 file(s) remainingremainingB/s) with 3 file(s) remaining (102.4 MiB/

In [34]:
!aws s3 ls s3://yifeim-code-data/criteo-rec/{pct}/

2018-10-04 17:49:17  143362983 test.idx
2018-10-04 17:49:17 15399403172 test.rec
2018-10-04 17:49:17 1668720811 test.txt.gz
2018-10-04 17:49:17  143360969 train.idx
2018-10-04 17:49:17 15402474916 train.rec
2018-10-04 17:49:26 1669613098 train.txt.gz
2018-10-04 17:49:33  143360304 val.idx
2018-10-04 17:49:34 15399459220 val.rec
2018-10-04 17:50:47 1668968200 val.txt.gz
2018-10-04 17:51:03     664116 vocab.txt
